In [ ]:
!pip install python-dotenv
!pip install selenium
!pip install pandas
!pip install reportlab

In [ ]:
# 基本ライブラリ
import time
import math
import os

# selenium
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.by import By

# pandas
import pandas

# reportlab
from reportlab.pdfgen import canvas
from reportlab.lib.units import mm
from reportlab.pdfbase.pdfmetrics import registerFont
from reportlab.pdfbase.ttfonts import TTFont


In [ ]:
# .env
%load_ext dotenv
%dotenv

#PATH_CHROMEDRIVER =("C:\\App\\ChromeDriver\\current\\chromedriver.exe")
PATH_CHROMEDRIVER = os.environ.get('PATH_CHROMEDRIVER')
URL_CODE39_MAKER = "https://barcode-place.azurewebsites.net/Barcode/code39"
path_download = os.getcwd() + os.path.sep + "work"
path_download

path_font = os.getcwd() + os.path.sep + "font"
FONT_BASIC = 'GenShinGothic'
registerFont(TTFont(FONT_BASIC, path_font + os.path.sep + 'GenShinGothic-Monospace-Medium.ttf'))
PDF_OUTPUT_FILENAME = 'Output.pdf'
PDF_A4_VERTICAL_W = 210 * mm
PDF_A4_VERTICAL_H = 297 * mm
# 画像実サイズ w 416px h 58px (208:29)
PDF_BARCODE_W = 75 * mm
PDF_BARCODE_H = PDF_BARCODE_W/208*29 # 10.5mm


In [ ]:
df_input = pandas.read_excel('MakeBarCode.xlsx', sheet_name='DATA',
            converters={
                0:str, # 品番
                1:str, # サイズ
                2:str, # カラー
                3:str, # 上代
                4:str, # SKU
                5:str, # バーコード上段
                6:str  # バーコード下段
            })
df_input

In [ ]:
options = webdriver.ChromeOptions()
options.add_experimental_option('prefs', {
    'download.default_directory' : path_download,
    "download.prompt_for_download": False
})

driver = webdriver.Chrome(executable_path=PATH_CHROMEDRIVER, options = options)
driver.get(URL_CODE39_MAKER)
time.sleep(2)

In [ ]:
pdf = canvas.Canvas(PDF_OUTPUT_FILENAME)
pdf.saveState()
pdf.setFont(FONT_BASIC, 18)
pdf.setPageSize((PDF_A4_VERTICAL_W, PDF_A4_VERTICAL_H))

rowIndex = 0
colIndex = 0
for row in df_input.itertuples():
    val_sku = row.SKU
    val_price = row.上代
    val_code39_01 = row.バーコード上段
    val_code39_02 = row.バーコード下段
    val_code39_01_image_filename = 'code39_'+ val_code39_01.replace(' ', '+') +'.png'
    #val_code39_01_image_filename = 'code39_'+ val_code39_01.replace(' ', '+') +'.eps'
    val_code39_02_image_filename = 'code39_'+ val_code39_02.replace(' ', '+') +'.png'
    #val_code39_02_image_filename = 'code39_'+ val_code39_02.replace(' ', '+') +'.eps'
    val_code39_01_image_filename_path = path_download + os.path.sep + val_code39_01_image_filename
    val_code39_02_image_filename_path = path_download + os.path.sep + val_code39_02_image_filename

    if not os.path.exists(val_code39_01_image_filename_path) or \
       not os.path.exists(val_code39_02_image_filename_path):
        # バーコード上段を入力
        input_fields = driver.find_elements(By.NAME, 'TextBoxBarcode00')
        for input_field in input_fields:
            input_field.send_keys(val_code39_01)
            break
        
        # バーコード下段を入力
        input_fields = driver.find_elements(By.NAME, 'TextBoxBarcode01')
        for input_field in input_fields:
            input_field.send_keys(val_code39_02)
            break

        # バーコード作成ボタンをクリック
        buttons = driver.find_elements(By.NAME, 'ButtonGene')
        for button in buttons:
            driver.execute_script('arguments[0].click();', button)
            break

        time.sleep(2)
        
        # バーコード上段画像DLボタンをクリック（すでにDL済みであればスキップ）
        if not os.path.exists(val_code39_01_image_filename_path):
            buttons = driver.find_elements(By.CLASS_NAME, 'btn-success')
            if len(buttons)==0:
                print("none")
            
            for button in buttons:
                if "send('button','code39_dl_png_01');" in button.get_attribute('onclick'):
                    driver.execute_script('arguments[0].click();', button)
                    break
        
        # バーコード下段画像DLボタンをクリック（すでにDL済みであればスキップ）
        if not os.path.exists(val_code39_02_image_filename_path):
            buttons = driver.find_elements(By.CLASS_NAME, 'btn-success')
            for button in buttons:
                if "send('button','code39_dl_png_02');" in button.get_attribute('onclick'):
                    driver.execute_script('arguments[0].click();', button)
                    break
            
        time.sleep(2)

        # 入力内容クリアボタンをクリック
        buttons = driver.find_elements(By.NAME, 'ButtonClear')
        for button in buttons:
            driver.execute_script('arguments[0].click();', button)
            break
    # ----------
    
    pos_x = 50 + colIndex*270
    pos_y = PDF_A4_VERTICAL_H-(50 + rowIndex*150)
    pdf.rect(pos_x-10, pos_y+25, 230, -135)
    pdf.drawString(pos_x, pos_y, val_sku)
    pos_y -= 20
    pdf.drawString(pos_x, pos_y, val_price + ' 円')
    pos_y -= 45
    pdf.drawInlineImage(val_code39_01_image_filename_path, pos_x, pos_y, PDF_BARCODE_W, PDF_BARCODE_H)
    pos_y -= 12 * mm
    pdf.drawInlineImage(val_code39_02_image_filename_path, pos_x, pos_y, PDF_BARCODE_W, PDF_BARCODE_H)
    
    if rowIndex < 4:
        if colIndex == 0:
            colIndex = 1
        else:
            rowIndex += 1
            colIndex = 0
    else:
        if colIndex == 0:
            colIndex = 1
        else:
            pdf.showPage() # 改ページ
            pdf.setFont(FONT_BASIC, 18)
            rowIndex = 0
            colIndex = 0
    
# ----------

# PDFファイル作成
pdf.save()